In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.16.166:39765
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
## path for mdules
import sys
sys.path.insert(0,"/home/jovyan/xscale")
import xscale


sys.path.insert(0,"/home/jovyan/eNATL60-plots-paper/common-lib/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dask.diagnostics import visualize
output_notebook()

%matplotlib inline

Loading BokehJS ...

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [4]:
zmap_grid = fs.get_mapper('pangeo-meom/eNATL60-I/eNATL60-mesh-mask')
ds_grid = xr.open_zarr(zmap_grid)


distributed.scheduler - INFO - Register tcp://10.32.24.4:42315
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.4:42315
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.24.5:42227
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.5:42227
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.28.4:34757
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.28.4:34757
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.28.5:37691
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.28.5:37691
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.35.4:32835
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.35.4:32835
distributed.core - INFO

In [5]:
zmap_u = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSU-1h')
ds_u = xr.open_zarr(zmap_u)


In [6]:
zmap_v = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSV-1h')
ds_v = xr.open_zarr(zmap_v)


distributed.scheduler - INFO - Remove worker tcp://10.32.23.4:46477
distributed.core - INFO - Removing comms to tcp://10.32.23.4:46477
distributed.scheduler - INFO - Remove worker tcp://10.32.23.5:46713
distributed.core - INFO - Removing comms to tcp://10.32.23.5:46713


In [7]:
navlat= ds_grid['nav_lat']
navlon= ds_grid['nav_lon']
e1u=ds_grid.e1u[0]
e1v=ds_grid.e1v[0]
e2u=ds_grid.e2u[0]
e2v=ds_grid.e2v[0]
ff=ds_grid.ff[0]


In [8]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=90, dim=['x', 'y'], n=[90, 90])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
def curl(u,v,e1v,e2u,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    #Computation of the vorticity divided by f
    fff=1/ff
    curl=(dxvt-dyut)*fff
    return curl

In [10]:
    print('Select dates in zarr')
    u=ds_u.sel(time_counter=slice('2010-03-01','2010-03-31'))['sozocrtx']
    v=ds_v.sel(time_counter=slice('2010-03-01','2010-03-31'))['somecrty']


Select dates in zarr


In [11]:
u

<xarray.DataArray 'sozocrtx' (time_counter: 744, y: 4729, x: 8354)>
dask.array<getitem, shape=(744, 4729, 8354), dtype=float32, chunksize=(24, 120, 120), chunktype=numpy.ndarray>
Coordinates:
    nav_lat        (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(296, 1045), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(120,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-03-01T00:30:00 ... 2010-03-31T23:30:00
Dimensions without coordinates: y, x
Attributes:
    cell_methods:        time: mean (interval: 40 s)
    interval_operation:  40 s
    interval_write:      1 h
    long_name:           ocean surface current along i-axis
    online_operation:    average
    units:               m/s

In [12]:
## chunksize = (24,120,120) => kernel restart

curl_surf=curl(u,v,e1v,e2u,ff)


In [ ]:
curl_surf.persist()

distributed.scheduler - INFO - Register tcp://10.32.29.3:37317
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.29.3:37317
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.29.4:41097
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.29.4:41097
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.23.3:35165
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.23.3:35165
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.23.2:34763
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.23.2:34763
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Remove worker tcp://10.32.46.5:46825
distributed.core - INFO - Removing comms to tcp://10.32.46.5:46825
distributed.scheduler - INFO - Remove

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof,CacheProfiler() as cprof:
    out = curl_surf.compute()

distributed.scheduler - INFO - Register tcp://10.32.25.2:37425
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.25.2:37425
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.25.3:36741
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.25.3:36741
distributed.core - INFO - Starting established connection
distributed.core - INFO - Event loop was unresponsive in Scheduler for 5.50s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Scheduler for 5.14s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Scheduler for 6.57s.  This is often caused by long-running GIL-holding functions or moving large chunks of 

In [ ]:
curl_surf.compute()